In [1]:
import torch
from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression

from concept_erasure import LeaceEraser

n, d, k = 2048, 128, 2

X, Y = make_classification(
    n_samples=n,
    n_features=d,
    n_classes=k,
    random_state=42,
)
X_t = torch.from_numpy(X)
Y_t = torch.from_numpy(Y)

# Logistic regression does learn something before concept erasure
real_lr = LogisticRegression(max_iter=1000).fit(X, Y)
beta = torch.from_numpy(real_lr.coef_)
assert beta.norm(p=torch.inf) > 0.1

eraser = LeaceEraser.fit(X_t, Y_t)
X_ = eraser(X_t)

# But learns nothing after
null_lr = LogisticRegression(max_iter=1000, tol=0.0).fit(X_.numpy(), Y)
beta = torch.from_numpy(null_lr.coef_)
assert beta.norm(p=torch.inf) < 1e-4

In [5]:
import pandas as pd
import pickle
df_19 = pd.read_csv("../data/cv-corpus-19.0-delta-2024-09-13/en/validated.tsv", delimiter="\t")
df_21 = pd.read_csv("../data/cv-corpus-21.0-delta-2025-03-14/en/validated.tsv", delimiter="\t")
df_17 = pd.read_csv('../data/cv-corpus-17.0-delta-2024-03-15/en/validated.tsv', delimiter="\t")

embeddings_19 = {}
with open('../data/cv_19.pkl', 'rb') as file:
    embeddings_19 = pickle.load(file)
embeddings_21 = {}
with open('../data/cv_21.pkl', 'rb') as file:
    embeddings_21 = pickle.load(file)
embeddings_17 = {}
with open('../data/cv_17.pkl', 'rb') as file:
    embeddings_17 = pickle.load(file)

In [ ]:
y_train= []
X_train= []
for idx, gender in enumerate(df_21.gender.tolist()):
    if gender in ["male_masculine", "female_feminine"]:
        y_train.append(gender)
        X_train.append(embeddings_21[df_21.path.tolist()[idx].split(".")[0]][0])
for idx, gender in enumerate(df_17.gender.tolist()):
    if gender in ["male_masculine", "female_feminine"]:
        y_train.append(gender)
        X_train.append(embeddings_17[df_17.path.tolist()[idx].split(".")[0]][0])
y_test= []
X_test= []
for idx, gender in enumerate(df_19.gender.tolist()):
    if gender in ["male_masculine", "female_feminine"]:
        y_test.append(gender)
        X_test.append(embeddings_19[df_19.path.tolist()[idx].split(".")[0]][0])

/var/folders/_l/m75yx49j3q98z5jct8l93xd40000gp/T/ipykernel_51442/3538986210.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_train.append(torch.tensor(embeddings_21[df_21.path.tolist()[idx].split(".")[0]][0]))
/var/folders/_l/m75yx49j3q98z5jct8l93xd40000gp/T/ipykernel_51442/3538986210.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_train.append(torch.tensor(embeddings_17[df_17.path.tolist()[idx].split(".")[0]][0]))
/var/folders/_l/m75yx49j3q98z5jct8l93xd40000gp/T/ipykernel_51442/3538986210.py:16: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(s

In [ ]:
len(X_train)

In [9]:
from sklearn.linear_model import LogisticRegression
import torch
from sklearn.metrics import accuracy_score

lr = LogisticRegression(max_iter=1000).fit(X_train, y_train)
beta = torch.from_numpy(lr.coef_)

beta.norm(p=torch.inf)

assert beta.norm(p=torch.inf) > 0.1
predict_train = lr.predict(X_train)
predict_test = lr.predict(X_test)
accuracy_score(y_train, predict_train), accuracy_score(y_test, predict_test)



/Users/rafkiep1/Desktop/AI_ethics_final/venv/lib/python3.10/site-packages/sklearn/utils/_array_api.py:839: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  array = numpy.asarray(array, order=order, dtype=dtype)


ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (1597,) + inhomogeneous part.

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

rf = RandomForestClassifier().fit(X_train, y_train)

predict_train = rf.predict(X_train)
predict_test = rf.predict(X_test)
accuracy_score(y_train, predict_train), accuracy_score(y_test, predict_test)



In [ ]:
len(y_train), len(y_test)

In [ ]:
from concept_erasure import LeaceEraser
eraser = LeaceEraser.fit(torch.tensor(X_train), torch.tensor([0  if y == "female_feminine" else 1 for y in y_train]))
X_test_debaised = eraser(torch.tensor(X_test))
X_train_debaised = eraser(torch.tensor(X_train))
assert(X_train_debaised != X_train)
rf2 = RandomForestClassifier().fit(X_train_debaised, y_train)
lr2 = LogisticRegression(max_iter=1000, tol=0.0).fit(X_train_debaised, y_train)

rf_predict_train_debiased = rf2.predict(X_train_debaised)
rf_predict_test_debiased = rf2.predict(X_test_debaised)

lr_predict_train_debaised = lr2.predict(X_train_debaised)
lr_predict_test_debaised = lr2.predict(X_test_debaised)
print(accuracy_score(y_train, rf_predict_train_debiased), accuracy_score(y_test, rf_predict_test_debiased))
print(accuracy_score(y_train, lr_predict_train_debaised), accuracy_score(y_test, lr_predict_test_debaised))
beta = torch.from_numpy(lr.coef_)
print(beta.norm(p=torch.inf))

# but now use these against some sort of task and see if gendered accuracy changes -- try accent classification maybe?